<center><img src="./../data/besm.png" alt="In the name of Allah" width="150"></center>

<style>
    
@font-face {font-family: "B Lotus"; src: url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.eot"); src: url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.woff") format("woff"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.svg#B Lotus") format("svg"); }


</style>

<center style ='font-family: "B Lotus";'>تمرین اول - درس پردازش زبان‌های طبیعی</center>
<center style ='font-family: "B Lotus";'>ترم ۱۴۰۱۱ - دانشکده مهندسی کامپیوتر</center>
<hr>
<center style ='font-family: "B Lotus";'>محمدحسین موثقی‌نیا</center>
<center style ='font-family: "B Lotus";'>۴۰۰۲۰۰۹۱۹</center>
<hr>
<center style ='font-family: "B Lotus";'><h2>استخراج داده از سایت فیلیمو</h2></center>
<hr>

<h2 style='direction:rtl;font-family: "B Lotus";'>§ مقدمه</h2>
<br>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">در این نوت‌بوک با استفاده از سه پکیج اصلی <code>requests</code>, <code>bs4</code>, <code>selenium</code> به استخراج کامنت‌های فیلم‌های موجود در صفحه اصلی هر ژانر در سایت فیلیمو پرداخته ایم. دلیل استفاده از پکیج <code>selenium</code> تجربه استفاده از آن همچنین نمایش نحوه استخراج بوده است، ولی عملا کار اصلی که همان استخراج کامنت‌ها بوده است با استفاده از پکیج <code>requests</code> بوده و همچنین کمک <code>bs4</code> به منظور تشخیص و استخراج محتوا از تگ های html انجام شده است.</div>
<br>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">در ادامه فرآیند به این صورت می باشد:
    <ul>
        <li>استخراج لینک تمامی فیلم‌های موجود در صفحه اصلی هر ژانر (به کمک پکیج <code>selenium</code>)</li>
        <li>استخراج نمایشی کامنت‌های فیلم‌ها (به کمک پکیج <code>selenium</code>)</li>
        <li>استخراج واقعی کامنت‌های لینک‌های فیلم به دست آمده در مرحله اول (به کمک پکیج <code>requests</code> و <code>bs4</code>)</li>
    </ul>
</div>
<hr>

<div style="direction: rtl; font-family: 'B Lotus'; font-size: 115%;">درج پکیج‌ها:</div>

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys, ActionChains

<h2 style='direction:rtl;font-family: "B Lotus";'>§ استخراج لینک ویدیوها</h2>

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">با توجه به این که یکی از پکیج&zwnj;های معروف استخراج اطلاعات از وب&zwnj;سایت&zwnj;ها پکیج <code><strong>Selenium</strong></code> می&zwnj;باشد، مبتنی بر آن تلاش کردم که به کمک آن محتوای کامنت&zwnj;ها را استخراج کنم.</div>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">استفاده از این پکیج صرفا برای تست و تصویرسازی عملکرد استخراج بوده است. علاوه براین لینک ویدیو‌ها نیز از این طریق استخراج شده است:</div>

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تابع <code>get_movies_url</code> به منظور استخراج url تمامی فیلم های موجود در صفحه اصلی هر ژانر مورد استفاده قرار می&zwnj;گیرد.</div>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">این تابع ابتدا صفحه مربوط به هر ژانر را باز کرده و تا جای ممکن اسکرول می کند، بعضی از صفحات به دلیل خاصیت lazy loading بعد از اسکرول تا انتها باز هم داده لود می کنند، برای این منظور یک حلقه تکرار شونده بررسی می&zwnj;کند که آیا باز هم جایی برای اسکرول هست یا نه. پس از اتمام اسکرول، تگ مربوط به لینک هر فیلم که یک تگ <code>a</code> می&zwnj;باشد و کلاس <code>thumb_wrapper</code> را دارد، انتخاب می&zwnj;شود. در نهایت از ویژگی <code>href</code> آن، لینک فیلم استخراج می&zwnj;شود. به همین شکل تمامی صفحات بررسی می&zwnj;شوند.</div>

In [52]:
def get_movies_url(url):
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url)

    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

    data = driver.page_source
    soup = BeautifulSoup(data, 'html.parser')
    movies = soup.select('a.thumb_wrapper')
    url_list = []
    for m in movies:
        if not(m.attrs['href'] in url_list):
            url_list.append(m.attrs['href'])
    return url_list

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">لیست مجموعه ژانرهای موجود در <code>genres</code> ذخیره می&zwnj;شود.</div>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">مبتنی بر آدرس پایه به همراه ژانر مورد نظر لینک تمامی صفحات اصلی ژانرها به دست می&zwnj;آید و در دیکشنری <code>genres_dic</code> با نام ژانر ذخیره می&zwnj;شود.</div>

In [40]:
genres = ["drama", "war", "adventure", "sci-fi", "romance", 
          "horror", "mystery", "family", "musical", "talk-show", 
          "short", "reality-tv", "history", "action", "fantasy", 
          "comedy", "western", "crime", "thriller", "sport", 
          "documentary", "music", "religious", "concert", 
          "biography", "theatre"]
genres_dic = {}
for g in genres:
    genres_dic[g] = f"https://www.filimo.com/tag/{g}"
genres_dic

{'drama': 'https://www.filimo.com/tag/drama',
 'war': 'https://www.filimo.com/tag/war',
 'adventure': 'https://www.filimo.com/tag/adventure',
 'sci-fi': 'https://www.filimo.com/tag/sci-fi',
 'romance': 'https://www.filimo.com/tag/romance',
 'horror': 'https://www.filimo.com/tag/horror',
 'mystery': 'https://www.filimo.com/tag/mystery',
 'family': 'https://www.filimo.com/tag/family',
 'musical': 'https://www.filimo.com/tag/musical',
 'talk-show': 'https://www.filimo.com/tag/talk-show',
 'short': 'https://www.filimo.com/tag/short',
 'reality-tv': 'https://www.filimo.com/tag/reality-tv',
 'history': 'https://www.filimo.com/tag/history',
 'action': 'https://www.filimo.com/tag/action',
 'fantasy': 'https://www.filimo.com/tag/fantasy',
 'comedy': 'https://www.filimo.com/tag/comedy',
 'western': 'https://www.filimo.com/tag/western',
 'crime': 'https://www.filimo.com/tag/crime',
 'thriller': 'https://www.filimo.com/tag/thriller',
 'sport': 'https://www.filimo.com/tag/sport',
 'documentary': 'h

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">با کمک تابع <code>get_movies_url</code> به ازای تمامی ژانرها صفحه اصلی آن&zwnj;ها بررسی شده و لینک تمامی فیلم&zwnj;های موجود در صفحه اصلی استخراج و در آدرس <code>movie.txt/.</code> به همراه ژانر آن ذخیره می&zwnj;شود.</div>

In [56]:
f = open("./../result/movies.txt", "a")
for item in genres_dic:
    tmp = get_movies_url(genres_dic[item])
    for m in tmp:
        f.write(f"{item}\t{m}\n")
f.close()

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">به منظور بررسی خروجی، محتویات فایل <code>movie.txt</code> را با استفاده از پکیج <code>pandas</code> می&zwnj;خوانیم.</div>

In [59]:
movies = pd.read_csv("./../result/movies.txt", names=["genre", "url"], delimiter='\t')
movies

,genre,url
0,drama,https://www.filimo.com/m/0v9if
1,drama,https://www.filimo.com/m/i3uga
2,drama,https://www.filimo.com/m/p8yhg
3,drama,https://www.filimo.com/m/vsucQ
4,drama,https://www.filimo.com/m/6lx40
...,...,...
4938,theatre,https://www.filimo.com/m/y078s
4939,theatre,https://www.filimo.com/m/0yiac
4940,theatre,https://www.filimo.com/m/ildr4
4941,theatre,https://www.filimo.com/m/eqwkc


<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">بررسی تعداد فیلم&zwnj;های موجود در هر ژانر</div>

In [64]:
num = {}
for m in genres:
    num[m] = 0
for m in movies["genre"]:
    num[m] += 1
num

{'drama': 752,
 'war': 82,
 'adventure': 483,
 'sci-fi': 113,
 'romance': 150,
 'horror': 200,
 'mystery': 165,
 'family': 234,
 'musical': 49,
 'talk-show': 70,
 'short': 152,
 'reality-tv': 35,
 'history': 111,
 'action': 569,
 'fantasy': 118,
 'comedy': 415,
 'western': 28,
 'crime': 381,
 'thriller': 236,
 'sport': 50,
 'documentary': 228,
 'music': 30,
 'religious': 42,
 'concert': 18,
 'biography': 208,
 'theatre': 24}

<h2 style='direction:rtl;font-family: "B Lotus";'>§ استخراج کامنت‌ها (نمایشی)</h2>
<br>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">این بخش صرفا یک نمایش استخراج می‌باشد و فرآیند اصلی در بخش بعدی انجام شده است که با استفاده از کتابخانه requests , bs4 می باشد.</div>

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تابع <code>extract_comments</code> تمامی کامنت&zwnj;های موجود در صفحه را استخراج می&zwnj;کند. این تابع در ورودی کنترل صفحه را گرفته و براساس کلاس تگ <code>p</code> مرتبط با متن کامنت‌ها، آن‌ها را تشخیص می‌دهد.</div>

In [ ]:
def extract_comments(driver):
    data = driver.page_source
    soup = BeautifulSoup(data, 'html.parser')
    comments = soup.select('p.comment-content')
    comment_texts = []
    for comment in comments:
        comment_texts.append(comment.text)
    return comment_texts

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تابع <code>number_of_comments</code> تعداد کامت&zwnj;های موجود در صفحه را شمارش می&zwnj;کند. این تابع در ورودی کنترل صفحه را گرفته و براساس کلاس تگ <code>p</code> مرتبط با متن کامنت‌ها، آن‌ها را تشخیص می‌دهد.</div>

In [ ]:
def number_of_comments(driver):
    data = driver.page_source
    soup = BeautifulSoup(data, 'html.parser')
    comments = soup.select('p.comment-content')
    return len(comments)

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تابع <code>get_comments</code> مجموعه دستورات لازم جهت استخراج تمامی کامنت&zwnj;ها را اجرا می کند که شامل صدا زدن دیگر توابع نیز می&zwnj;شود. در نهایت خروجی آن شامل مجموعه تمامی کامنت&zwnj;ها و همچنین کنترل صفحه وب موجود می&zwnj;باشد.</div>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">این تابع در ورودی آدرس سایت مدنظر را گرفته و براساس تگ <code>button</code> و آیدی مربوط به بارگزاری بیشتر کامنت&zwnj;ها، آن را تشخیص داد و با اسکرول کردن صفحه و کلیک بر روی این کلید، تمامی کامنت&zwnj;ها را بارگزاری می&zwnj;کند. در هر مرحله نیز تعداد کامنت&zwnj;ها بررسی می&zwnj;شود تا نشان دهنده پیشرفت برنامه باشد. در نهایت کنترل صفحه به تابع <code>extract_comments</code> داده می&zwnj;شود تا تمامی کامنت&zwnj;ها را استخراج کند.</div>

In [90]:
def get_comments(url):
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    button = driver.find_element(By.ID, "comments-loadmore")
    print("Website has been loaded!")
    print("Extraction has been started...")
    num = 0
    while True:
        time.sleep(2)
        try:
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            time.sleep(0.5)
            button = driver.find_element(By.ID, "comments-loadmore")
            ActionChains(driver).click(button).perform()
            time.sleep(1)
            tmp = number_of_comments(driver) - num
            num = number_of_comments(driver)
            print(f"total: {num}\tnew: +{tmp}")
        except:
            print("DONE!")
            break
    return driver, extract_comments(driver)

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تست تابع <code>get_comments</code> برای آدرس &quot;<a href="https://www.filimo.com/m/119698" rel="noopener noreferrer" target="_blank"><u>سریال روزی روزگاری مریخ</u></a>&quot;</div>

In [18]:
driver, comments = get_comments("https://www.filimo.com/m/119698")

Website has been loaded!
Extract Started...
total: 44	new: +44
total: 65	new: +21
total: 86	new: +21
total: 107	new: +21
total: 128	new: +21
total: 149	new: +21
total: 170	new: +21
total: 191	new: +21
total: 212	new: +21
total: 233	new: +21
total: 254	new: +21
total: 257	new: +3
DONE!


<h2 style='direction:rtl;font-family: "B Lotus";'>§ استخراج کامنت‌ها (اصلی)</h2>
<br>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">استخراج اصلی کامنت‌ها در این بخش انجام شده است:</div>

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تابع <code>extract_comments</code> با گرفتن متغیر <code>soup</code> که آبجکتی از جنس <code>BeautifulSoup</code> می باشد، تمامی تگ&zwnj;های <code>p</code> ای که دارای کلاس <code>comment-content</code> هستند را استخراج می&zwnj;کند. سپس یک پاکسازی اولیه روی متن&zwnj;های انجام داده و آن را خروجی می&zwnj;دهد.</div>

In [5]:
def extract_comments(soup):
    comment_elements = soup.find_all("p", class_="comment-content")
    
    comments = []
    # extract text of comments
    for comment in comment_elements:
        comments.append(comment.text)
    
    # delete all '\n' from data
    for i in range(len(comments)):
        comments[i] = comments[i].replace('\n', '')
    
    return comments

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تابع <code>get_all_comments</code> هسته اصلی استخراج کننده متن کامنت هاست. روند به این صورت است که هر صفحه فیلم براساس آدرس هایی که قبلا استخراج شده است، به این تابع داده می&zwnj;شود و تابع مدنظر ابتدا صفحه را لود کرده و سپس کامنت‌های صفحه را استخراج و آدرس کلید &quot;نمایش بیشتر&quot; برای کامنت‌ها را به دست آورده و به آن آدرس درخواست می‌دهد و بقیه کامنت‌ها لود می‌شود، این عمل بارها و بارها تکرار می‌شود تا زمانی که دیگر کلید &quot;نمایش بیشتر&quot; وجود نداشته باشد به این معنی که تمامی کامنت‌ها لود شده است. در هر مرحله چون صفحه جدیدی لود می‌شود کامنت‌های قبلی در آن نیست بنابراین کافیست کامنت‌های موجود در صفحه لود شده ذخیره شود.<div>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">در بین درخواست‌ها نیز به صورت رندم بین <code>0 تا 1.5</code> ثانیه تاخیر ایجاد می‌کنیم که توسط فایروال‌های سایت مورد نظر بسته نشویم.<div>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">این تابع در خروجی لیست کامنت‌های فیلم مورد نظر و تایتل آن را بر می‌گرداند.<div>

In [6]:
def get_all_comments(url):
    
    comments = []
    
    print("> Extraction ...")
    data = requests.get(url)
    soup = BeautifulSoup(data.text, 'html.parser')
    title = soup.title.text.replace('\n', '').replace(' |', '|').replace('| ', '|')
    print(f"> Movie: {title}")
    
    d = ["NULL"]
    comment_url = url
    
    while d != []:
        data = requests.get(comment_url)
        soup = BeautifulSoup(data.text, 'html.parser')
        d = soup.find_all("button", class_="request-link comments-loadmore is-ajax-button")
        
        # ectract comments
        comments.extend(extract_comments(soup))
        
        # sleep about 0 to 1.5 sec
        time.sleep(np.random.uniform(0, 1.5))
        
        # get next comments url
        if d != []:
            u = d[0]["data-href"]
            comment_url = f"https://www.filimo.com{u}"
        else:
            break
    
    print("> Done!")
    
    return comments, title

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">لود آدرس&zwnj;های استخراج شده از فیلم&zwnj;های ژانرهای مختلف</div>

In [7]:
movies = pd.read_csv("./../result/movies.txt", names=["genre", "url"], delimiter='\t')
movies

,genre,url
0,drama,https://www.filimo.com/m/0v9if
1,drama,https://www.filimo.com/m/i3uga
2,drama,https://www.filimo.com/m/p8yhg
3,drama,https://www.filimo.com/m/vsucQ
4,drama,https://www.filimo.com/m/6lx40
...,...,...
4938,theatre,https://www.filimo.com/m/y078s
4939,theatre,https://www.filimo.com/m/0yiac
4940,theatre,https://www.filimo.com/m/ildr4
4941,theatre,https://www.filimo.com/m/eqwkc


<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تست تابع <code>get_all_comments</code> برای یک آدرس دلخواه از فیلم&zwnj;ها<div>

In [8]:
get_all_comments("https://www.filimo.com/m/2le7o")

> Extraction ...
> Movie: فیلم نشانی جمکران|دانلود و تماشای آنلاین با بالاترین کیفیت| فیلیمو
> Done!


(['عالی', 'عالی', 'بینظیر'],
 'فیلم نشانی جمکران|دانلود و تماشای آنلاین با بالاترین کیفیت| فیلیمو')

<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">تابع <code>extraction</code> یک تابع اجرایی است، به این معنی که با استفاده از تابع <code>get_all_comments</code> که هسته استخراج می‌باشد و ارسال آدرس فیلم&zwnj;ها به این تابع، روند استخراج را اجرا می‌کند. این تابع دو پارامتر <code>num_until</code> و <code>batch</code> را دریافت می&zwnj;کند که به ترتیب، شماره ردیفی از فیلم که باید از آن شروع کند و تعداد فیلم هایی که باید بررسی کند، می‌باشد.</div>
<div style="direction:rtl; font-family: 'B Lotus'; font-size: 115%;">در نهایت خروجی‌ها به صورت سه تایی (ژانر، تایتل فیلم، کامنت) در یک فایل در آدرس <code>result/comments.txt/../.</code> ذخیره می‌شود.</div>

In [12]:
def extraction(num_until, batch, file_suffix):
    f = open(f"./../result/comments_{file_suffix}.txt", "a")

    num = 1
    total = len(movies["genre"])

    for g, url in zip(movies["genre"], movies["url"]):

        if num <= num_until:
            num += 1
            continue

        if num > num_until + batch:
            break
        
        print(f"# Genre: {g}")
        
        start = time.time()
        comments, title = get_all_comments(url)
        end = time.time()
        print(f"#[{num:04d}/{total}] Execution time: {end-start:.5f} sec")
        
        num += 1
        
        for m in comments:
            f.write(f"{g}\t{title}\t{m}\n")

        print(f'# Saved in "./../result/comments_{file_suffix}.txt"')
        print("--------------")
        
    f.close()

In [22]:
file_num = 3
done_until = 4500
batch_size = 443
extraction(num_until=done_until, batch=batch_size, file_suffix=file_num)

# Genre: documentary
> Extraction ...
> Movie: سریال رویارویی با علی (Facing Ali 2009)|دانلود و تماشای آنلاین با بالاترین کیفیت|فیلیمو
> Done!
#[4501/4943] Execution time: 1.38877 sec
# Saved in "./../result/comments_3.txt"
--------------
# Genre: documentary
> Extraction ...
> Movie: فیلم دیگو مارادونا (Diego Maradona 2019)|دوبله فارسی|فیلیمو
> Done!
#[4502/4943] Execution time: 17.65774 sec
# Saved in "./../result/comments_3.txt"
--------------
# Genre: documentary
> Extraction ...
> Movie: سریال فیت|دانلود و تماشای آنلاین با بالاترین کیفیت| فیلیمو
> Done!
#[4503/4943] Execution time: 3.06748 sec
# Saved in "./../result/comments_3.txt"
--------------
# Genre: documentary
> Extraction ...
> Movie: سریال به نام خلق|دانلود و تماشای آنلاین با بالاترین کیفیت| فیلیمو
> Done!
#[4504/4943] Execution time: 2.91245 sec
# Saved in "./../result/comments_3.txt"
--------------
# Genre: documentary
> Extraction ...
> Movie: سریال خارج از دید 2 : نفوذ|دانلود و تماشای آنلاین با بالاترین کیفیت| فیلیمو